In [1]:
import pandas as pan
import numpy as np

#loading csv
dlrPitches = pan.read_csv("C:/####/dlr-pitches.csv")

p=len(dlrPitches['Location'])


In [2]:
#Filling up the NaN in 'Location' as values set in the previous rows

count =0.0
while(count < p):
    if(pan.isnull(dlrPitches['Location'][count])):
        dlrPitches['Location'][count]
        dlrPitches['Location'][count]=dlrPitches['Location'][count-1]
    count = count + 1

c:\users\furqan\appdata\local\programs\python\python35-32\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [3]:
#Viewing the Location column of dlrPitches
dlrPitches['Location']

0           Kilbogget Park
1           Kilbogget Park
2           Kilbogget Park
3           Kilbogget Park
4           Kilbogget Park
5           Kilbogget Park
6           Kilbogget Park
7           Kilbogget Park
8           Kilbogget Park
9           Kilbogget Park
10          Kilbogget Park
11          Kilbogget Park
12          Kilbogget Park
13          Kilbogget Park
14          Kilbogget Park
15          Kilbogget Park
16          Kilbogget Park
17          Kilbogget Park
18          Kilbogget Park
19          Kilbogget Park
20         Shanganagh Park
21         Shanganagh Park
22         Shanganagh Park
23         Shanganagh Park
24         Shanganagh Park
25         Shanganagh Park
26       Shanganagh Cliffs
27       Shanganagh Cliffs
28       Shanganagh Castle
29    Rockfield Park(NTPL)
              ...         
31    Rockfield Park(NTPL)
32           Dunedin (jun)
33      Soldiers & Sailors
34              Meadowvale
35              Meadowvale
36              Meadowvale
3

In [27]:
#dlrPitches

In [4]:
#Rounding off the Latitude and Longitude values in dlrPitches to 5 decimal points
dlrPitches['Latitude']=round(dlrPitches['Latitude'],5)
dlrPitches['Longitude']=round(dlrPitches['Longitude'],5)
#dlrPitches

In [5]:
#Parsing the xml data 
import xml.etree.cElementTree as et
#Creating a fcc_data dataframe which would contain the parsed XML data
fcc_data=pan.DataFrame()
#Returns the node value if the xml element is not empty else returns None
def getValue(node):
    if node is not None:
       return node.text
    else:
       return None



#this function returns xml data loaded into fcc_data
def mainfunction():
    parsedXML = et.parse('C:/####/xmldata.xml')
    #gets the root XML element
    root=parsedXML.getroot()
    dfcols=['typef','name','location','lat','long']
    df_xml=pan.DataFrame(columns=dfcols)
    
    for node in root.findall('./Playing_Pitches-table/Playing_Pitches'):
        typef=node.find('FACILITY_TYPE')
        name=node.find('FACILITY_NAME')
        location=node.find('LOCATION')
        lat=node.find('LAT')
        long=node.find('LONG')
        
        df_xml=df_xml.append(
            pan.Series([getValue(typef),getValue(name),getValue(location),
                        getValue(lat),getValue(long)],index=dfcols),ignore_index=True)
    
    return pan.DataFrame(df_xml)
    
fcc_data=mainfunction()
    



In [6]:
fcc_data.dtypes
#data was of object type changing it to float
fcc_data.lat.astype(float)     
fcc_data['lat']=fcc_data.lat.astype(float)
fcc_data['long']=fcc_data.long.astype(float)
#rounding off the columns 'lat' and 'long' to 5 decimal places
fcc_data['lat']=round(fcc_data['lat'],5)
fcc_data['long']=round(fcc_data['long'],5)
#renaming columns long and lat to Longitude and Latitude for consistency
fcc_data.rename(columns={'long': 'Longitude', 'lat': 'Latitude'}, inplace=True)


In [7]:
#dccplayingpitchesp20120816-1550.csv
#Loading CSV
dccPitches = pan.read_csv("C:/####/dccplayingpitchesp20120816-1550.csv")
dccPitches
#we see an unnamed column with just NaN values
#Checking the number of columns
len(dccPitches.columns)

#dropping the column with NaN values
dccPitches=dccPitches.drop(dccPitches.columns[4],axis=1)
#checking the no. of datasets , now one less than the one containing just NaNs
len(dccPitches.columns)
#changing the String object column PARK to lower case
dccPitches['PARK']=dccPitches['PARK'].str.lower()

In [8]:
#DCC data does not have the geographic locations
#Loading another dataset to extract the geo locations
townlandData = pan.read_csv("C:/###/Townlands__OSi_National_Placenames_Gazetteer.csv")
#rounding off the geographic locations to 5 decimal places
townlandData['X']=round(townlandData['X'],5)
townlandData['Y']=round(townlandData['Y'],5)
#Taking a subset of townlandData
#Considering only the data related to county DUBLIN
townlandData=townlandData[townlandData['County']=='DUBLIN']
#Renaming columns to match the geo locations in other datasets
townlandData.rename(columns={'X': 'Longitude', 'Y': 'Latitude'}, inplace=True)

c:\users\###\local\programs\python\python35-32\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#Changing the datatype of column 'English_Name' to type string
#Changing the case to lower
townlandData['English_Name'] = townlandData['English_Name'].astype(str)
townlandData['English_Name'] = townlandData['English_Name'].str.lower()


In [10]:
#Renaming English_Name to PARK
townlandData.rename(columns={'English_Name': 'PARK'}, inplace=True)


In [11]:
#merging the datasets of dccPitches and townlandData through left join
#The mergedDataDCC contains now the columns of DCC and townlandData including the Latitude, Longitude
mergedDataDCC=pan.merge(dccPitches,townlandData,on='PARK',how='left')


In [12]:
#renaming the location of dlrPitches to PARK
dlrPitches.rename(columns={'Location': 'PARK'}, inplace=True)


In [13]:
#Merging dlrPitches and mergedDataDCC based on the keys Latitude and Longitude. Merging all the columns using 
#outer join
dlrdccMerged = dlrPitches.merge(mergedDataDCC,on=['Latitude','Longitude'],how='outer')

In [14]:
#Merging dlrdccMerged and fcc_data through outer join based on geographic locations (latitude and longitude)
dlrdccfccMerged = dlrdccMerged.merge(fcc_data,on=['Latitude','Longitude'],how='outer')

In [15]:
#taking only rows with non-NaN values in Latitude column
dlrdccfccMerged=dlrdccfccMerged[np.isfinite(dlrdccfccMerged['Latitude'])]

In [16]:
#taking rows with non-NaN values in Longitude columnn
dlrdccfccMerged=dlrdccfccMerged[np.isfinite(dlrdccfccMerged['Longitude'])]

In [20]:
#Viewing the resultant dlrdccfccMerged dataset with non-empty Latitude and Longitude values
#Uncomment the below code to view it
#uncomment -> #dlrdccfccMerged

In [17]:
#checking for columns with NaN values

dlrdccfccMerged.isnull().sum()


PARK_x                       144
Number                       145
Size                         146
Latitude                       0
Longitude                      0
PARK_y                       156
AREA                         156
CLUBNAME                     156
LEAGUE                       156
OBJECTID                     156
County                       156
Contae                       156
Local_Government_Area        156
Limistéar_Rialtas_Áitiúil    204
Classification               156
Cineál                       156
Gaeltacht                    156
Town_Classification          204
ID                           156
Irish_Name                   156
Foirm_Ghinideach             204
Alternative_Name             204
IG_E                         156
IG_N                         156
ITM_E                        156
ITM_N                        156
Irish_Validation             204
Legislation                  204
Validated_By                 156
Date_                        204
Comment_  

In [18]:
#Delete columns containing only NaN values
dlrdccfccMerged=dlrdccfccMerged.dropna(axis=1, how='all')

In [19]:
#getting duplicated Latitude and Longitude rows and putting it into dataset 'duplicates'
duplicates=dlrdccfccMerged[dlrdccfccMerged.duplicated(['Latitude', 'Longitude'],keep=False)]

In [20]:
#checking the NaN values in each row of the duplicates
#This is done in order to select the duplicates' row with least NaN values
duplicates.isnull().sum(axis=1).tolist()

[6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6]

In [21]:
#Since all rows contained equal number of NaN values
#dropping rows with duplicates keeping first row in the dlrdccfccMerged dataset
dlrdccfccMerged=dlrdccfccMerged.drop_duplicates(subset=['Latitude', 'Longitude'], keep='first')

In [22]:
#merging park related columns into the PARK_x
#since they convey same information in two different columns
dlrdccfccMerged['PARK_x']=dlrdccfccMerged['PARK_x'].fillna(dlrdccfccMerged['PARK_y'])

In [23]:
#merging park related columns into the PARK_x
#since they convey same information in two different columns
dlrdccfccMerged['PARK_x']=dlrdccfccMerged['PARK_x'].fillna(dlrdccfccMerged['name'])

In [24]:
#deleting columns with redundant information
del dlrdccfccMerged['PARK_y']
del dlrdccfccMerged['name']

In [25]:
# delete columns
#These are deleted for having two few information
#for example if the the following rows contain 156 NaN out of 168 total values
# AREA                     156
# CLUBNAME                 156
# LEAGUE                   156
# Local_Government_Area    156
# Classification           156
# ID                       156
# Irish_Name               156
# Validated_By             156
del dlrdccfccMerged['IG_E']
del dlrdccfccMerged['IG_N']
del dlrdccfccMerged['ITM_E']
del dlrdccfccMerged['ITM_N']
del dlrdccfccMerged['County']
del dlrdccfccMerged['Contae']
del dlrdccfccMerged['OBJECTID']
del dlrdccfccMerged['Cineál']
del dlrdccfccMerged['Gaeltacht']
del dlrdccfccMerged['AREA']
del dlrdccfccMerged['CLUBNAME']
del dlrdccfccMerged['LEAGUE']
del dlrdccfccMerged['Local_Government_Area']
del dlrdccfccMerged['Classification']
del dlrdccfccMerged['ID']
del dlrdccfccMerged['Irish_Name']
del dlrdccfccMerged['Validated_By']
#Change code below

In [26]:
#merging park related columns
dlrdccfccMerged['typef']=dlrdccfccMerged['typef'].fillna(dlrdccfccMerged['Number'])

In [27]:
#deleting column with redundant values
del dlrdccfccMerged['Number']

In [28]:
#Changing the datatype of typef column to string
dlrdccfccMerged['typef'] = dlrdccfccMerged['typef'].astype(str)

In [29]:
#Replace numeric data with NaN
#This is done so as to remove unwanted data from columns
dlrdccfccMerged['typef'] = dlrdccfccMerged['typef'].str.replace('\d+', 'NaN')

In [30]:
#Replace Numeric data with NaN
#This is done so as to remove unwanted data from columns
dlrdccfccMerged['Size'] = dlrdccfccMerged['Size'].str.replace('\d+', 'NaN')

In [31]:
#Renaming columns
dlrdccfccMerged.rename(columns={'PARK_x': 'Park_Name', 'typef': 'Pitch(Type)'}, inplace=True)

In [32]:
dlrdccfccMerged.isnull().sum()

Park_Name        0
Size           110
Latitude         0
Longitude        0
Pitch(Type)      0
location        95
dtype: int64

In [33]:
#outputting a csv file
dlrdccfccMerged.to_csv('C:/###/MergedData.csv', sep=',')

In [34]:
dlrdccfccMerged

,Park_Name,Size,Latitude,Longitude,Pitch(Type),location
0,Kilbogget Park,Snr,53.25724,-6.14067,NaN,NaN
1,Kilbogget Park,SSG,53.25761,-6.13988,NaN,NaN
2,Kilbogget Park,SSG,53.25784,-6.13927,NaN,NaN
3,Kilbogget Park,SSG,53.25710,-6.13909,NaN,NaN
4,Kilbogget Park,SGG,53.25667,-6.14013,NaN,NaN
5,Kilbogget Park,Snr,53.25660,-6.13815,NaN,NaN
6,Kilbogget Park,Snr,53.25624,-6.13876,NaN,NaN
7,Kilbogget Park,Snr,53.25605,-6.13972,NaN,NaN
8,Kilbogget Park,Snr,53.25567,-6.14181,NaN,NaN
9,Kilbogget Park,SSG,53.25858,-6.14245,NaN,NaN
